# Clinical Data Integration

**Goal**: Merge MIMIC-III clinical tables with waveform signal segments to create a cohesive ground truth dataset.

**Inputs**:
- MIMIC-III Clinical Database (PATIENTS.csv, ADMISSIONS.csv, DIAGNOSES_ICD.csv)
- Signal metadata from Sprint 1 (signal_clinical_integrated_dataset.parquet)

**Outputs**:
- `signal_clinical_integrated_dataset.parquet`: Single source of truth with all segment-level metadata

**Pipeline**:
1. Load MIMIC clinical tables (with caching)
2. Extract cardiometabolic labels (Diabetes, Hypertension, Obesity)
3. Link waveform segments to clinical admissions (Option B)
4. Extract and process demographics (age, sex, BMI)
5. Compute Charlson Comorbidity Index
6. Assemble final integrated dataset

## Setup

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import logging

# 1. Set project root
PROJECT_ROOT = Path.cwd().parent
os.chdir(PROJECT_ROOT)

sys.path.insert(0, str(PROJECT_ROOT / "colab_src"))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("colab_src exists:", (PROJECT_ROOT / "colab_src").exists())
print("data_pipeline exists:", (PROJECT_ROOT / "colab_src" / "data_pipeline").exists())


PROJECT_ROOT: c:\Developments\cardiometabolic-risk-colab
colab_src exists: True
data_pipeline exists: True


In [3]:
# Import  modules from colab_src.data_pipeline
from data_pipeline import (
    MIMICClinicalExtractor, 
    ClinicalLabelExtractor, 
    WaveformClinicalLinker, 
    DemographicProcessor, 
    DatasetAssembler
)

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration
MIMIC_CLINICAL_DIR = Path('data/raw')  # MIMIC CSV files location
CACHE_DIR = Path('data/cache')
OUTPUT_DIR = Path('data/processed')

# Ensure output directories exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
CACHE_DIR.mkdir(parents=True, exist_ok=True)

print("✅ All imports successful")
print(f"\n📁 Configuration:")
print(f"   Project Root:       {PROJECT_ROOT}")
print(f"   MIMIC Clinical Dir: {MIMIC_CLINICAL_DIR}")
print(f"   Cache Dir:          {CACHE_DIR}")
print(f"   Output Dir:         {OUTPUT_DIR}")

# Check for MIMIC files
if MIMIC_CLINICAL_DIR.exists():
    csv_files = list(MIMIC_CLINICAL_DIR.glob('*.csv'))
    print(f"\n✅ MIMIC directory found with {len(csv_files)} CSV files:")
    for f in sorted(csv_files):
        print(f"   - {f.name}")
else:
    print(f"\n⚠️  Warning: MIMIC directory not found at {MIMIC_CLINICAL_DIR}")

✅ All imports successful

📁 Configuration:
   Project Root:       c:\Developments\cardiometabolic-risk-colab
   MIMIC Clinical Dir: data\raw
   Cache Dir:          data\cache
   Output Dir:         data\processed

✅ MIMIC directory found with 6 CSV files:
   - ADMISSIONS.csv
   - CHARTEVENTS.csv
   - DIAGNOSES_ICD.csv
   - ICUSTAYS.csv
   - LABEVENTS.csv
   - PATIENTS.csv


## Step 1: Extract MIMIC Clinical Tables

In [4]:
# Inspect MIMIC CSV files to see actual column names
print("📋 Inspecting MIMIC CSV file structures:\n")

# Check PATIENTS.csv
patients_file = MIMIC_CLINICAL_DIR / 'PATIENTS.csv'
if patients_file.exists():
    print(f"✅ PATIENTS.csv found")
    df_sample = pd.read_csv(patients_file, nrows=0)  # Read just header
    print(f"   Columns: {list(df_sample.columns)}\n")
else:
    print(f"⚠️  PATIENTS.csv not found\n")

# Check ADMISSIONS.csv
admissions_file = MIMIC_CLINICAL_DIR / 'ADMISSIONS.csv'
if admissions_file.exists():
    print(f"✅ ADMISSIONS.csv found")
    df_sample = pd.read_csv(admissions_file, nrows=0)
    print(f"   Columns: {list(df_sample.columns)}\n")
else:
    print(f"⚠️  ADMISSIONS.csv not found\n")

# Check DIAGNOSES_ICD.csv
diagnoses_file = MIMIC_CLINICAL_DIR / 'DIAGNOSES_ICD.csv'
if diagnoses_file.exists():
    print(f"✅ DIAGNOSES_ICD.csv found")
    df_sample = pd.read_csv(diagnoses_file, nrows=0)
    print(f"   Columns: {list(df_sample.columns)}\n")
else:
    print(f"⚠️  DIAGNOSES_ICD.csv not found\n")

📋 Inspecting MIMIC CSV file structures:

✅ PATIENTS.csv found
   Columns: ['row_id', 'subject_id', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag']

✅ ADMISSIONS.csv found
   Columns: ['row_id', 'subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admission_location', 'discharge_location', 'insurance', 'language', 'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis', 'hospital_expire_flag', 'has_chartevents_data']

✅ DIAGNOSES_ICD.csv found
   Columns: ['row_id', 'subject_id', 'hadm_id', 'seq_num', 'icd9_code']



In [5]:
# Load MIMIC clinical tables with caching
extractor = MIMICClinicalExtractor(MIMIC_CLINICAL_DIR, cache_dir=CACHE_DIR)
patients_df, admissions_df, diagnoses_df = extractor.load_clinical_tables(use_cache=True)

# Validate
is_valid = extractor.validate_clinical_tables()

if is_valid:
    stats = extractor.get_summary_statistics()
    print(f"\n✅ Clinical tables loaded and validated")
    print(f"   Summary: {stats}")
else:
    print(f"\n⚠️  Some validation issues found. Review warnings above.")

2026-01-11 19:05:13,968 - data_pipeline.mimic_clinical_extractor - INFO - MIMICClinicalExtractor initialized
2026-01-11 19:05:13,970 - data_pipeline.mimic_clinical_extractor - INFO -   Clinical dir: data\raw
2026-01-11 19:05:13,971 - data_pipeline.mimic_clinical_extractor - INFO -   Cache dir: data\cache
2026-01-11 19:05:13,972 - data_pipeline.mimic_clinical_extractor - INFO - 📦 Loading from cache: data\cache\mimic_clinical_tables.pkl
2026-01-11 19:05:14,003 - data_pipeline.mimic_clinical_extractor - INFO - ✅ Loaded from cache:
2026-01-11 19:05:14,006 - data_pipeline.mimic_clinical_extractor - INFO -    PATIENTS: 100 records
2026-01-11 19:05:14,007 - data_pipeline.mimic_clinical_extractor - INFO -    ADMISSIONS: 129 records
2026-01-11 19:05:14,009 - data_pipeline.mimic_clinical_extractor - INFO -    DIAGNOSES_ICD: 1761 records
2026-01-11 19:05:14,010 - data_pipeline.mimic_clinical_extractor - INFO - 
🔍 Validating clinical tables...
2026-01-11 19:05:14,017 - data_pipeline.mimic_clinical


✅ Clinical tables loaded and validated
   Summary: {'n_patients': 100, 'n_admissions': 129, 'n_diagnosis_records': 1761, 'n_unique_patients_in_admissions': 100, 'n_unique_diagnoses': 581, 'date_range_admissions': (Timestamp('2102-08-29 07:15:00'), Timestamp('2202-10-03 01:45:00'))}


## Step 2: Extract Cardiometabolic Labels and Charlson Index

In [6]:
# Extract labels
label_extractor = ClinicalLabelExtractor()
labels_df = label_extractor.extract_labels_per_admission(diagnoses_df)

# Compute Charlson Index
cci_df = label_extractor.compute_charlson_index(diagnoses_df)

print(f"\n✅ Labels and Charlson Index extracted")
print(f"   Labels shape: {labels_df.shape}")
print(f"   CCI shape:    {cci_df.shape}")

2026-01-11 19:05:29,935 - data_pipeline.clinical_label_extractor - INFO - 🏥 Extracting cardiometabolic labels from ICD-9 codes...
2026-01-11 19:05:29,967 - data_pipeline.clinical_label_extractor - INFO - ✅ Extracted labels for 129 admissions
2026-01-11 19:05:29,979 - data_pipeline.clinical_label_extractor - INFO -    Diabetes:         48 ( 37.2%)
2026-01-11 19:05:29,982 - data_pipeline.clinical_label_extractor - INFO -    Hypertension:     73 ( 56.6%)
2026-01-11 19:05:29,990 - data_pipeline.clinical_label_extractor - INFO -    Obesity:           4 (  3.1%)
2026-01-11 19:05:29,998 - data_pipeline.clinical_label_extractor - INFO -    Multi-morbid:     40 ( 31.0%)
2026-01-11 19:05:30,000 - data_pipeline.clinical_label_extractor - INFO -    Healthy (0,0,0):    46 ( 35.7%)
2026-01-11 19:05:30,001 - data_pipeline.clinical_label_extractor - INFO - 📊 Computing Charlson Comorbidity Index...
2026-01-11 19:05:30,103 - data_pipeline.clinical_label_extractor - INFO - ✅ Computed CCI for 129 admissio


✅ Labels and Charlson Index extracted
   Labels shape: (129, 4)
   CCI shape:    (129, 2)


## Step 3: Load Signal Metadata from Sprint 1

In [12]:
# Load signal metadata from Sprint 1
signal_metadata_path = OUTPUT_DIR / 'sprint1_metadata.parquet'
signal_metadata_df = pd.read_parquet(signal_metadata_path)

print(f"✅ Loaded signal metadata from Sprint 1")
print(f"   Path:  {signal_metadata_path}")
print(f"   Shape: {signal_metadata_df.shape}")
print(f"   Columns: {list(signal_metadata_df.columns)}")
print(f"\n   Sample (first 3 rows):")
print(signal_metadata_df.head(3))

✅ Loaded signal metadata from Sprint 1
   Path:  data\processed\sprint1_metadata.parquet
   Shape: (4417, 11)
   Columns: ['record_name', 'subject_id', 'segment_idx', 'fs', 'sqi_score', 'quality_grade', 'snr_db', 'perfusion_index', 'channel_name', 'global_segment_idx', 'batch_num']

   Sample (first 3 rows):
                record_name subject_id  segment_idx   fs  sqi_score  \
0  p00/p000052/3533390_0004    p000052            0  125   0.893482   
1  p00/p000052/3533390_0004    p000052            1  125   0.888996   
2  p00/p000052/3238451_0005    p000052            0  125   0.888845   

  quality_grade     snr_db  perfusion_index channel_name  global_segment_idx  \
0     Excellent  39.255102     3.938813e+06        PLETH                   0   
1     Excellent  38.742355     3.663113e+06        PLETH                   1   
2     Excellent  38.725109     1.637093e+06        PLETH                   2   

   batch_num  
0        1.0  
1        1.0  
2        1.0  


In [14]:
# Investigate the signal metadata source - extract real MIMIC IDs from record paths
print("🔍 INVESTIGATING SIGNAL METADATA - EXTRACTING REAL MIMIC IDs:\n")

print(f"Signal metadata columns: {list(signal_metadata_df.columns)}")
print(f"\nSample record_name values (first 5):")
for i, record in enumerate(signal_metadata_df['record_name'].head(5)):
    print(f"   {i+1}. {record}")

print(f"\n--- UNDERSTANDING RECORD PATH STRUCTURE ---")
print(f"Format appears to be: p00/p######/########_####")
print(f"  - p00: folder prefix")
print(f"  - p######: extracted subject ID (e.g., p000052 → 52)")
print(f"  - ########_####: record ID or timestamp")

print(f"\n--- CHECKING FOR DATETIME IN RECORD NAMES ---")
# Check if record names contain datetime info like 'p000020-2183-04-28-17-47'
has_datetime = signal_metadata_df['record_name'].str.contains('-').any()
print(f"Records contain datetime: {has_datetime}")

if has_datetime:
    print(f"\nExample with datetime:")
    datetime_records = signal_metadata_df[signal_metadata_df['record_name'].str.contains('-')]
    if len(datetime_records) > 0:
        print(f"  {datetime_records['record_name'].iloc[0]}")

print(f"\n⚠️  CRITICAL FINDING:")
print(f"The 'subject_id' in signal_metadata_df (52, 53, ...) are EXTRACTED IDs")
print(f"They do NOT match MIMIC's actual SUBJECT_ID (10006, 10011, ...)")
print(f"\nPossible solutions:")
print(f"  1. Look for a mapping file in data/processed/ or data/raw/")
print(f"  2. Check if these are synthetic/demo records (not real MIMIC)")
print(f"  3. Check MIMIC waveform database metadata")

# List all files in data folder to find mapping
print(f"\n📁 Searching for mapping files in data folders...")
import os
data_folders = ['data/raw', 'data/processed', 'data']
for folder in data_folders:
    folder_path = Path(folder)
    if folder_path.exists():
        print(f"\n  {folder}:")
        for file in sorted(folder_path.glob('*')):
            if file.is_file():
                print(f"    - {file.name}")


STEP 3b: EXTRACT SUBJECT IDs FROM SIGNAL RECORDS & FIND INTERSECTION

🔧 Extracting subject IDs from signal metadata...

✅ Extracted subject IDs from all 4417 records
   Sample records:
                record_name subject_id  demo_id  real_subject_id  \
0  p00/p000052/3533390_0004    p000052       52            10052   
1  p00/p000052/3533390_0004    p000052       52            10052   
2  p00/p000052/3238451_0005    p000052       52            10052   
3  p00/p000052/3238451_0005    p000052       52            10052   
4  p00/p000052/3238451_0005    p000052       52            10052   

   subject_id_int  
0              52  
1              52  
2              52  
3              52  
4              52  

INTERSECTION ANALYSIS

📊 Dataset Overview:
   Clinical subjects (MIMIC CSV): 100
   Clinical subject ID range: 10006 - 44228
   Signal metadata records: 4417
   Unique demo_ids in signals: 130
   Unique real_subject_ids in signals: 130
   Signal demo_ids range: 52 - 4833

✅ INTERSECTI

In [20]:

# Check for NATURAL overlap between waveform subject IDs and clinical CSV subject IDs
print("="*80)
print("CHECKING NATURAL OVERLAP - WAVEFORMS vs CLINICAL CSVs")
print("="*80)

# Get subject IDs from both sources
waveform_subjects = set(signal_metadata_df['subject_id'].str.replace('p', '').astype(int).unique())
clinical_subjects = set(admissions_df['subject_id'].unique())

print(f"\n📊 SUBJECT ID RANGES:")
print(f"   Waveform subjects:")
print(f"      Count: {len(waveform_subjects)}")
print(f"      Range: {min(waveform_subjects)} - {max(waveform_subjects)}")
print(f"      Sample IDs: {sorted(list(waveform_subjects))[:20]}")

print(f"\n   Clinical CSV subjects (from ADMISSIONS):")
print(f"      Count: {len(clinical_subjects)}")
print(f"      Range: {min(clinical_subjects)} - {max(clinical_subjects)}")
print(f"      Sample IDs: {sorted(list(clinical_subjects))[:20]}")

# Check for overlap
overlap = waveform_subjects & clinical_subjects

print(f"\n🔍 OVERLAP ANALYSIS:")
print(f"   Subjects in both datasets: {len(overlap)}")

if len(overlap) > 0:
    print(f"   ✅ FOUND {len(overlap)} overlapping subjects!")
    print(f"   Sample overlapping IDs: {sorted(list(overlap))[:20]}")
    
    # Count segments for each overlapping subject
    overlap_segments_per_subject = signal_metadata_df[
        signal_metadata_df['subject_id'].str.replace('p', '').astype(int).isin(overlap)
    ].groupby('subject_id').size()
    
    print(f"\n   Segments per overlapping subject:")
    print(f"      Total segments in overlap: {overlap_segments_per_subject.sum()}")
    print(f"      Mean per subject: {overlap_segments_per_subject.mean():.1f}")
    print(f"      Median per subject: {overlap_segments_per_subject.median():.0f}")
    print(f"      Max per subject: {overlap_segments_per_subject.max()}")
    
    # Show top 10 overlapping subjects by segment count
    print(f"\n   Top 10 overlapping subjects by segment count:")
    top_overlap = overlap_segments_per_subject.nlargest(10)
    for subject_id, count in top_overlap.items():
        sid = int(subject_id.replace('p', ''))
        print(f"      Subject {sid:>5}: {count:>4} segments")
else:
    print(f"   ❌ NO OVERLAP FOUND")
    print(f"   Waveform IDs and Clinical CSV IDs are from completely different patient populations")
    print(f"\n   💡 Recommendation:")
    print(f"      - The demo clinical CSVs (10001-10100) don't match waveform subjects")
    print(f"      - Need full MIMIC-III clinical database OR")
    print(f"      - Need mapping file to link demo waveform records to full database patients")

print(f"\n" + "="*80)


CHECKING NATURAL OVERLAP - WAVEFORMS vs CLINICAL CSVs

📊 SUBJECT ID RANGES:
   Waveform subjects:
      Count: 130
      Range: 52 - 4833
      Sample IDs: [52, 107, 109, 124, 154, 160, 188, 222, 262, 301, 333, 357, 402, 518, 523, 543, 550, 600, 605, 618]

   Clinical CSV subjects (from ADMISSIONS):
      Count: 100
      Range: 10006 - 44228
      Sample IDs: [10006, 10011, 10013, 10017, 10019, 10026, 10027, 10029, 10032, 10033, 10035, 10036, 10038, 10040, 10042, 10043, 10044, 10045, 10046, 10056]

🔍 OVERLAP ANALYSIS:
   Subjects in both datasets: 0
   ❌ NO OVERLAP FOUND
   Waveform IDs and Clinical CSV IDs are from completely different patient populations

   💡 Recommendation:
      - The demo clinical CSVs (10001-10100) don't match waveform subjects
      - Need full MIMIC-III clinical database OR
      - Need mapping file to link demo waveform records to full database patients



## Step 4: Link Waveforms to Clinical Admissions 

In [17]:
# Link segments to admissions using parallel processing
linker = WaveformClinicalLinker(n_jobs=-1, batch_size=1000)
linked_segments_df = linker.link_segments_to_admissions(
    signal_metadata_df,
    admissions_df,
    n_jobs=-1,
    show_progress=True
)

print(f"\n✅ Segments linked to admissions")
print(f"   Linked segments shape: {linked_segments_df.shape}")
print(f"   New columns: {linked_segments_df.columns.difference(signal_metadata_df.columns).tolist()}")

2026-01-11 22:37:47,262 - data_pipeline.waveform_to_clinical_linker - INFO - 🔗 Linking 282 segments to admissions (n_jobs=-1)
2026-01-11 22:37:47,408 - data_pipeline.waveform_to_clinical_linker - INFO -    Processing 1 batches of size 1000...


2026-01-11 22:37:47,262 - data_pipeline.waveform_to_clinical_linker - INFO - 🔗 Linking 282 segments to admissions (n_jobs=-1)
2026-01-11 22:37:47,408 - data_pipeline.waveform_to_clinical_linker - INFO -    Processing 1 batches of size 1000...


🔧 LINKING INTERSECTION SEGMENTS TO CLINICAL ADMISSIONS

   Original signal metadata: 4417 segments
   Intersection segments: 282 segments
   Subjects: 1

   Prepared dataframe for linking:
      Rows: 282
      Columns: ['record_name', 'subject_id', 'demo_id', 'real_subject_id', 'subject_id_int', 'fs', 'sqi_score', 'quality_grade', 'snr_db', 'perfusion_index', 'channel_name', 'global_segment_idx', 'batch_num', 'segment_id']
      Subject ID range: 10124 - 10124

📡 Linking segments to admissions...



2026-01-11 22:37:47,262 - data_pipeline.waveform_to_clinical_linker - INFO - 🔗 Linking 282 segments to admissions (n_jobs=-1)
2026-01-11 22:37:47,408 - data_pipeline.waveform_to_clinical_linker - INFO -    Processing 1 batches of size 1000...


🔧 LINKING INTERSECTION SEGMENTS TO CLINICAL ADMISSIONS

   Original signal metadata: 4417 segments
   Intersection segments: 282 segments
   Subjects: 1

   Prepared dataframe for linking:
      Rows: 282
      Columns: ['record_name', 'subject_id', 'demo_id', 'real_subject_id', 'subject_id_int', 'fs', 'sqi_score', 'quality_grade', 'snr_db', 'perfusion_index', 'channel_name', 'global_segment_idx', 'batch_num', 'segment_id']
      Subject ID range: 10124 - 10124

📡 Linking segments to admissions...



2026-01-11 22:37:55,808 - data_pipeline.waveform_to_clinical_linker - INFO - ✅ Linking complete:
2026-01-11 22:37:55,811 - data_pipeline.waveform_to_clinical_linker - INFO -    Linked:      282 segments
2026-01-11 22:37:55,812 - data_pipeline.waveform_to_clinical_linker - INFO -    Unlinked:      0 segments (orphans)


2026-01-11 22:37:47,262 - data_pipeline.waveform_to_clinical_linker - INFO - 🔗 Linking 282 segments to admissions (n_jobs=-1)
2026-01-11 22:37:47,408 - data_pipeline.waveform_to_clinical_linker - INFO -    Processing 1 batches of size 1000...


🔧 LINKING INTERSECTION SEGMENTS TO CLINICAL ADMISSIONS

   Original signal metadata: 4417 segments
   Intersection segments: 282 segments
   Subjects: 1

   Prepared dataframe for linking:
      Rows: 282
      Columns: ['record_name', 'subject_id', 'demo_id', 'real_subject_id', 'subject_id_int', 'fs', 'sqi_score', 'quality_grade', 'snr_db', 'perfusion_index', 'channel_name', 'global_segment_idx', 'batch_num', 'segment_id']
      Subject ID range: 10124 - 10124

📡 Linking segments to admissions...



2026-01-11 22:37:55,808 - data_pipeline.waveform_to_clinical_linker - INFO - ✅ Linking complete:
2026-01-11 22:37:55,811 - data_pipeline.waveform_to_clinical_linker - INFO -    Linked:      282 segments
2026-01-11 22:37:55,812 - data_pipeline.waveform_to_clinical_linker - INFO -    Unlinked:      0 segments (orphans)



✅ LINKING COMPLETE
   Linked segments shape: (282, 15)

   hadm_id linking success: 282/282 (100.0%)

✅ hadm_id linking successful!


## Step 5: Process Demographics (Age, Sex, BMI)

In [19]:
# Inspect linked_segments_df structure before demographics processing
print("📋 Inspecting linked_segments_df structure:\n")
print(f"   Shape: {linked_segments_df.shape}")
print(f"   Columns: {list(linked_segments_df.columns)}")
print(f"\n   First few rows:")
print(linked_segments_df.head(2))
print(f"\n   Data types:")
print(linked_segments_df.dtypes)

# Fix: Create segment_id column from global_segment_idx (required by DemographicProcessor)
# Also convert subject_id from string to integer if needed
print("\n\n✅ Preparing data for demographics processing...")
linked_segments_df = linked_segments_df.copy()
linked_segments_df['segment_id'] = linked_segments_df['global_segment_idx']

# Convert subject_id from string (p000052) to integer (52)
linked_segments_df['subject_id'] = linked_segments_df['subject_id'].str.replace('p', '').astype(int)

print(f"   Added 'segment_id' column")
print(f"   Converted 'subject_id' to integer format")
print(f"\n   Updated columns: {list(linked_segments_df.columns)}")
print(f"   Sample:")
print(linked_segments_df[['segment_id', 'subject_id', 'hadm_id']].head(3))

📋 Inspecting linked_segments_df structure:

   Shape: (282, 15)
   Columns: ['record_name', 'subject_id', 'demo_id', 'real_subject_id', 'subject_id_int', 'fs', 'sqi_score', 'quality_grade', 'snr_db', 'perfusion_index', 'channel_name', 'global_segment_idx', 'batch_num', 'segment_id', 'hadm_id']
   Data types:
record_name            object
subject_id              int32
demo_id                 int64
real_subject_id         int64
subject_id_int          int32
fs                      int64
sqi_score             float64
quality_grade          object
snr_db                float64
perfusion_index       float64
channel_name           object
global_segment_idx      int64
batch_num             float64
segment_id              int64
hadm_id                 int64
dtype: object

✅ Preparing data for demographics processing...
   Added 'segment_id' column

   subject_id info:
      Type: int32
      Sample values: [10124, 10124, 10124]
      Is numeric: True

   Updated columns: ['record_name', 'subje

In [12]:
# Process demographics with parallel processing
demo_processor = DemographicProcessor(n_jobs=-1)
demographics_df = demo_processor.process_demographics(
    linked_segments_df,
    patients_df,
    admissions_df,
    n_jobs=-1
)

print(f"\n✅ Demographics processed")
print(f"   Demographics shape: {demographics_df.shape}")
print(f"   Columns: {list(demographics_df.columns)}")
print(f"\n   Sample (first 3 rows):")
print(demographics_df.head(3))

2026-01-11 18:40:52,335 - data_pipeline.demographic_and_bmi_processor - INFO - 👤 Processing demographics for 4417 segments
2026-01-11 18:40:52,339 - data_pipeline.demographic_and_bmi_processor - INFO -    Processing 4417 segments in parallel...
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
2026-01-11 18:40:58,451 - data_pipeline.demographic_and_bmi_processor - WARNING - ⚠️  All BMI values missing. Using default: 25.0
2026-01-11 18:40:58,453 - data_pipeline.demographic_and_bmi_processor - INFO - ✅ Demographics processed:
2026-01-11 18:40:58,454 - data_pipeline.demographic_and_bmi_processor - INFO -    Valid age:          0 segments
2026-01-11 18:40:58,454 - data_pipeline.demographic_and_bmi_processor - INFO -    Missing age:     4417 segments (will exclude)
2026-01-11 18:40:58,457 - data_pipeline.demographic_and_bmi_processor - INFO -    


✅ Demographics processed
   Demographics shape: (4417, 7)
   Columns: ['segment_id', 'subject_id', 'hadm_id', 'age', 'sex', 'bmi', 'is_bmi_missing']

   Sample (first 3 rows):
   segment_id  subject_id hadm_id  age  sex   bmi  is_bmi_missing
0           0          52    None  NaN  NaN  25.0             NaN
1           1          52    None  NaN  NaN  25.0             NaN
2           2          52    None  NaN  NaN  25.0             NaN


## Step 6: Assemble Final Dataset

In [16]:
print("📋 Inspecting all dataframes before assembly:\n")

print(f"signal_metadata_df:")
print(f"   Shape: {signal_metadata_df.shape}")
print(f"   Columns: {list(signal_metadata_df.columns)}")
print(f"   Has 'segment_id': {'segment_id' in signal_metadata_df.columns}")

print(f"\nlinked_segments_df:")
print(f"   Shape: {linked_segments_df.shape}")
print(f"   Columns: {list(linked_segments_df.columns)}")
print(f"   Has 'segment_id': {'segment_id' in linked_segments_df.columns}")

print(f"\nlabels_df:")
print(f"   Shape: {labels_df.shape}")
print(f"   Columns: {list(labels_df.columns)}")

print(f"\ndemographics_df:")
print(f"   Shape: {demographics_df.shape}")
print(f"   Columns: {list(demographics_df.columns)}")

print(f"\ncci_df:")
print(f"   Shape: {cci_df.shape}")
print(f"   Columns: {list(cci_df.columns)}")

# Fix: Add segment_id to signal_metadata_df if not present
if 'segment_id' not in signal_metadata_df.columns:
    print("\n✅ Adding 'segment_id' to signal_metadata_df...")
    signal_metadata_df = signal_metadata_df.copy()
    signal_metadata_df['segment_id'] = signal_metadata_df.get('global_segment_idx', 
                                                               range(len(signal_metadata_df)))
    print(f"   signal_metadata_df now has {len(signal_metadata_df.columns)} columns")


📋 Inspecting all dataframes before assembly:

signal_metadata_df:
   Shape: (4417, 12)
   Columns: ['record_name', 'subject_id', 'segment_idx', 'fs', 'sqi_score', 'quality_grade', 'snr_db', 'perfusion_index', 'channel_name', 'global_segment_idx', 'batch_num', 'segment_id']
   Has 'segment_id': True

linked_segments_df:
   Shape: (4417, 13)
   Columns: ['record_name', 'subject_id', 'segment_idx', 'fs', 'sqi_score', 'quality_grade', 'snr_db', 'perfusion_index', 'channel_name', 'global_segment_idx', 'batch_num', 'hadm_id', 'segment_id']
   Has 'segment_id': True

labels_df:
   Shape: (129, 4)
   Columns: ['hadm_id', 'diabetes', 'hypertension', 'obesity']

demographics_df:
   Shape: (4417, 7)
   Columns: ['segment_id', 'subject_id', 'hadm_id', 'age', 'sex', 'bmi', 'is_bmi_missing']

cci_df:
   Shape: (129, 2)
   Columns: ['hadm_id', 'cci_score']


In [18]:
# DEBUG: Investigate hadm_id issue BEFORE assembly
print("🔍 DEBUGGING hadm_id issue:\n")

print(f"linked_segments_df 'hadm_id' column:")
print(f"   Non-null count: {linked_segments_df['hadm_id'].notna().sum()} / {len(linked_segments_df)}")
print(f"   Null count: {linked_segments_df['hadm_id'].isna().sum()}")
print(f"   Unique values: {linked_segments_df['hadm_id'].nunique()}")
if linked_segments_df['hadm_id'].notna().any():
    print(f"   Sample values: {linked_segments_df['hadm_id'].dropna().head(5).tolist()}")
else:
    print(f"   Sample values: ALL NONE/NULL")

print(f"\nlabels_df 'hadm_id' column:")
print(f"   Count: {len(labels_df)}")
print(f"   Unique hadm_ids: {labels_df['hadm_id'].nunique()}")
print(f"   Sample hadm_ids: {labels_df['hadm_id'].head(5).tolist()}")
print(f"   Data type: {labels_df['hadm_id'].dtype}")

print(f"\ndemographics_df 'hadm_id' column:")
print(f"   Non-null count: {demographics_df['hadm_id'].notna().sum()} / {len(demographics_df)}")
print(f"   Null count: {demographics_df['hadm_id'].isna().sum()}")
print(f"   Sample values: {demographics_df['hadm_id'].dropna().head(5).tolist() if demographics_df['hadm_id'].notna().any() else 'ALL NONE'}")
print(f"   Data type: {demographics_df['hadm_id'].dtype}")

print(f"\ncci_df 'hadm_id' column:")
print(f"   Count: {len(cci_df)}")
print(f"   Unique hadm_ids: {cci_df['hadm_id'].nunique()}")
print(f"   Data type: {cci_df['hadm_id'].dtype}")

# Check for subject_id format mismatch
print(f"\n--- CHECKING SUBJECT_ID FORMATS ---")
print(f"\nlinked_segments_df 'subject_id':")
print(f"   Type: {linked_segments_df['subject_id'].dtype}")
print(f"   Sample values: {linked_segments_df['subject_id'].head(3).tolist()}")

print(f"\nadmissions_df 'subject_id':")
print(f"   Type: {admissions_df['subject_id'].dtype}")
print(f"   Sample values: {admissions_df['subject_id'].head(3).tolist()}")

# Check if any subjects match
matched_subjects = set(linked_segments_df['subject_id'].unique()) & set(admissions_df['subject_id'].unique())
print(f"\n--- SUBJECT OVERLAP ANALYSIS ---")
print(f"   Subjects in linked_segments_df: {linked_segments_df['subject_id'].nunique()}")
print(f"   Subjects in admissions_df: {admissions_df['subject_id'].nunique()}")
print(f"   Subjects in common: {len(matched_subjects)}")

if len(matched_subjects) == 0:
    print(f"\n🚨 NO MATCHING SUBJECTS!")
    print(f"   This explains why hadm_id is all None.")
    print(f"   The signal metadata and MIMIC admissions don't overlap.")
else:
    print(f"\n✓ {len(matched_subjects)} subjects match between datasets.")
    print(f"   If hadm_id is still None, the linking algorithm needs fixing.")

🔍 DEBUGGING hadm_id issue:

linked_segments_df 'hadm_id' column:
   Non-null count: 0 / 4417
   Null count: 4417
   Unique values: 0
   Sample values: ALL NONE/NULL

labels_df 'hadm_id' column:
   Count: 129
   Unique hadm_ids: 129
   Sample hadm_ids: [100375, 100969, 101361, 102203, 103379]
   Data type: int64

demographics_df 'hadm_id' column:
   Non-null count: 0 / 4417
   Null count: 4417
   Sample values: ALL NONE
   Data type: object

cci_df 'hadm_id' column:
   Count: 129
   Unique hadm_ids: 129
   Data type: int64

--- CHECKING SUBJECT_ID FORMATS ---

linked_segments_df 'subject_id':
   Type: int32
   Sample values: [52, 52, 52]

admissions_df 'subject_id':
   Type: int64
   Sample values: [10006, 10011, 10013]

--- SUBJECT OVERLAP ANALYSIS ---
   Subjects in linked_segments_df: 130
   Subjects in admissions_df: 100
   Subjects in common: 0

🚨 NO MATCHING SUBJECTS!
   This explains why hadm_id is all None.
   The signal metadata and MIMIC admissions don't overlap.


In [17]:
# Assemble all components
# Use linked_segments_df (which has hadm_id) instead of signal_metadata_df
assembler = DatasetAssembler(output_dir=OUTPUT_DIR)
final_dataset = assembler.assemble(
    linked_segments_df,  # Use linked_segments_df (has hadm_id + segment_id)
    linked_segments_df,  # Pass the same dataframe for consistency
    labels_df,
    demographics_df,
    cci_df,
    output_filename='signal_clinical_integrated_dataset.parquet'
)

print(f"\n✅ Final dataset assembled and saved")
print(f"   Shape: {final_dataset.shape}")
print(f"   Columns: {list(final_dataset.columns)}")

2026-01-11 18:47:21,901 - data_pipeline.dataset_assembly - INFO - 
🔀 ASSEMBLING FINAL DATASET
2026-01-11 18:47:21,902 - data_pipeline.dataset_assembly - INFO -    Input segments: 4417
2026-01-11 18:47:21,910 - data_pipeline.dataset_assembly - INFO -    After hadm_id merge: 4417 segments


KeyError: 'hadm_id'

## Verification and Summary

In [ ]:
print("\n" + "="*80)
print("PHASE 0 COMPLETION SUMMARY")
print("="*80)

print(f"\n✅ DATASET CREATED: signal_clinical_integrated_dataset.parquet")
print(f"   Location: {OUTPUT_DIR / 'signal_clinical_integrated_dataset.parquet'}")
print(f"   Size: {final_dataset.shape[0]} rows × {final_dataset.shape[1]} columns")

print(f"\n✅ KEY STATISTICS:")
print(f"   Subjects:          {final_dataset['subject_id'].nunique()}")
print(f"   Admissions:        {final_dataset['hadm_id'].nunique()}")
print(f"   Segments:          {len(final_dataset)}")

print(f"\n✅ LABEL DISTRIBUTION:")
print(f"   Diabetes:          {final_dataset['diabetes'].sum()} ({100*final_dataset['diabetes'].mean():.1f}%)")
print(f"   Hypertension:      {final_dataset['hypertension'].sum()} ({100*final_dataset['hypertension'].mean():.1f}%)")
print(f"   Obesity:           {final_dataset['obesity'].sum()} ({100*final_dataset['obesity'].mean():.1f}%)")

print(f"\n✅ DEMOGRAPHICS:")
print(f"   Age:               {final_dataset['age'].mean():.1f} ± {final_dataset['age'].std():.1f} years")
print(f"   Sex (M/F ratio):   {(final_dataset['sex']==1).sum()}/{(final_dataset['sex']==0).sum()}")
print(f"   BMI median:        {final_dataset['bmi'].median():.1f} kg/m²")
print(f"   BMI imputed:       {final_dataset['is_bmi_missing'].sum()} ({100*final_dataset['is_bmi_missing'].mean():.1f}%)")

print(f"\n✅ SIGNAL QUALITY:")
print(f"   Mean SQI:          {final_dataset['sqi_score'].mean():.3f}")
print(f"   Mean SNR:          {final_dataset['snr_db'].mean():.1f} dB")

print(f"\n" + "="*80)
print("PHASE 0 COMPLETE ✅")
print("Ready for Phase 1: Feature Engineering")
print("="*80)

## Next Steps

Phase 0 is now complete. The dataset `signal_clinical_integrated_dataset.parquet` is ready for:

1. **Phase 1**: Feature Engineering
   - Extract HRV, morphology, and clinical context features
   - Perform multicollinearity analysis
   - Select final feature set

2. **Phase 2**: Model Training
   - Train CNN encoder (1D-ResNet)
   - Train XGBoost classifiers (5 conditions)
   - Train fusion model (SQI-gated ensemble)

3. **Phase 3**: Evaluation & Interpretability
   - Compute clinical metrics (AUPRC, sensitivity, NPV)
   - Fairness analysis
   - SHAP explanations

4. **Phase 4**: API Deployment
   - Export models (ONNX, Pickle)
   - FastAPI application
   - Docker containerization